In [11]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from pandas import Series, DataFrame
import glob
import os

In [12]:
allFiles = sorted(glob.glob("*.txt"),key=os.path.getmtime) # includes all files into a single stucture,
# Sorted by modification time so that the first measurement is first...
print(allFiles)

['SHGData6_070217.itx.txt', 'SHGData7__070217.txt', 'SHGData8__070217.txt', 'SHGData9__070217.txt', 'SHGData10__070217.txt', 'SHGData11__070217.txt', 'SHGData12__070217.txt', 'SHGData13__070217.txt', 'SHGData14__070217.txt', 'SHGData15__070217.txt', 'SHGData16__070217.txt']


In [13]:
frame = pd.DataFrame()
df = pd.DataFrame()

In [14]:
for i,file_ in enumerate(allFiles):
#    f = open(file_,'r')
#     for j,line in enumerate(f):
#         if j == 1:
#             date = line[14:22]
#             break
    df = pd.read_csv(file_,index_col = None, header=None, skiprows = 10, skipfooter = 1, delimiter = '\t')
#     frame[file_] = df[5]
    df['file']=file_
    df['timeDelay']= df[1]
    df['PDvalue'] = df[3]
#     df['date']=date
    if i < 1:
        frame = df
    else:
        frame = frame.append(df)

/Users/sanjaykhatri/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:7: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skip_footer; you can avoid this warning by specifying engine='python'.


In [15]:
c = 3*10**11; #Speed of Light in mm/s, Going to use this to convert spatial information to time. 
T1 = (499*0.000125 + 0.0229)*2*25.4/(10**-12*c)
def ConvertToTimeDelay(x,itr,offset):
    c = 3*10**11
    timeDelay = []
    for i in range(0,len(x)):
        timedelay = ((x[i]*itr + offset)*25.4*2/(10**-12*c))-T1
        timeDelay.append(timedelay)
    return(timeDelay)

In [16]:
times = pd.Series()
PDvalues = pd.Series()
AvgPD50 = pd.Series()
itr = 0.000125
offset = 0.023
for i in range(0,900):
    time = frame[1][i]
    PDvalue = frame[3][i]
    times = times.append(time)
    PDvalues = PDvalues.append(PDvalue)
#print(len(PDvalues))
    
times = times.values.reshape(900,len(allFiles))
PDvalues = PDvalues.values.reshape(900,len(allFiles)) 
timeDelay = ConvertToTimeDelay(times,itr,offset) # function converts to timedelay

#PDvalues = np.array(PDvalues.tolist()).reshape((900,len(allFiles)-16))
#PDvalues = PDvalues.astype(np.float)
#print(np.where(PDvalues=='-0.-0220')) # array 5,32 Used to find where this funky element was
#AvgPD50 = PDvalues.mean(axis = 1) 

PDvalues[5][-2] = -0.0220 # An error popped up because this looked like: -'0.-220'
PDvalues = PDvalues.astype(np.float) #Ensures all the values are floats so that I can plot them!
AvgPD50 = PDvalues.mean(axis = 1) #average of all the datasets
#timeDelay = ConvertToTimeDelay(times, itr, offset)

In [17]:
extraticks = [-10,-9,-8,-7,-6,-5,-4, -3, -2, -1, 1, 2, 3, 4,5,6,7,8,9,10]
plt.plot(timeDelay,AvgPD50,'b')
plt.xlabel('Stage Delay (ps)')
plt.ylabel('PD Signal (arb)')
plt.grid(True)
plt.xticks(list(plt.xticks()[0]) + extraticks)
plt.xlim(-11,11)
plt.show()
#plt.savefig('PD_SignalVsDelay_7_2_2017.png')